In [1]:
import pandas as pd
review_df=pd.read_csv('labeledTrainData.tsv', header=0, sep="\t", quoting=3)

C:\Users\taeeu\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\taeeu\anaconda3\lib\site-packages\numpy\.libs\libopenblas.el2c6ple4zyw3eceviv3oxxgrn2nrfm2.gfortran-win_amd64.dll
C:\Users\taeeu\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import re

# HTML태그가 담겨있음을 확인. 이는 크롤링을 통해서 가져올 때 HTML을 이용하기 때문에
# 분석에 필요없는 HTML을 제거해준다.
review_df['review'] = review_df['review'].str.replace('<br />',' ')

# 특수문자에 대해서도 공백으로 바꾸는 클렌징 작업을 진행
review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]"," ",x))

## 1. 품사태깅 수행 - penn_to_wn(tag)

In [3]:
from nltk.corpus import wordnet as wn

#NLTK PennTreebank Tag기반 WordNet의 품사 Tag으로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB

## 2. 문장->단어 토큰화->품사 태깅
## SentiSynset클래스 생성
## Polarity Score 합산

In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [5]:
def swn_polarity(text):
    #감성 지수 초기화
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    
    #분해된 문장별로 단어 토큰 -> 품사 태깅 후 sentiSynset 생성 -> 감성 지수 합산
    for raw_sentence in raw_sentences:
        #NLTK 기반의 품사 태깅 문장 추출
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:
            
            #WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN,wn.ADJ,wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word,pos=wn_tag)
            if not lemma:
                continue
                    #어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력하여 Synset 객체를 생성
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
                    #sentiwordnet의 감성 단어 분석으로 감성 synset 추출
                    #모든 단어에 대해 긍정 감성 지수는 +로, 부정 감성 지수는 -로 합산하여 감성 지수 계산
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            tokens_count += 1
                    
            
    if not tokens_count:
        return 0
            
            
            
    #총 score가 0이상이면 긍정 1, 그렇지 않으면 부정 0 반환
    if sentiment >= 0:
        return 1
            
    return 0

## 3. swn_polarity(text) 함수를 개별 문서에 적용

In [6]:
review_df['preds'] = review_df['review'].apply(lambda x : swn_polarity(x))

KeyboardInterrupt: 

In [ ]:
y_target = review_df['sentiment'].values
preds = review_df['preds'].values